In [10]:
from functools import reduce
from py_ecc import bn128
from py_ecc.bn128 import G1, G2, bn128_curve, curve_order, multiply, eq, neg, field_modulus, pairing
from sage.all import *

F = GF(curve_order)
F_x.<x> = F[] # polynomial over finite field

A = Matrix(F, [
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,0,1,0]
])

B = Matrix(F, [
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,1]
])

C = Matrix(F,[
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [0,1,0,0,0,0,0,0]
])

# x*2 + x + 3

x =  F.random_element()
y =  F.random_element()
z =  F.random_element()
u =  F.random_element()

out = x * y * z * u
v1 = x*y
v2 = z*u

# create the witness vector
w = vector(F, [1, out, x, y, z, u, v1, v2])


assert Matrix(C * w) == Matrix(A * w).elementwise_product(Matrix(B * w))

def trusted_setup(n, t_x):
    encrypted_G1 = list()
    encrypted_G2 = list()
    t_x_enc = list()

    tau = F.random_element()
    pow_of_tau = 1  # tau ** 0
    t_x_eval = t_x(tau)

    # pow_tau_C = list()
    for i in range(0, n):
        temp = multiply(G1, int(pow_of_tau))
   
        encrypted_G1.append(temp)
        encrypted_G2.append(multiply(G2, int(pow_of_tau)))

        if i <= n-2:
            t_x_enc.append(multiply(temp, int(t_x_eval)))

        pow_of_tau = pow_of_tau*tau

    return encrypted_G1, encrypted_G2, t_x_enc

def generate_target_polynomial(n):
    # Create a polynomial of the form (x - 1)(x - 2)(x - 3)...(x - n)
    x = var('x')
    res = F_x(prod([(x - i) for i in range(1, n+1)]))
    for i in range(1, n+1):
        assert res(i) == 0, "Invalid target polynom"
    return res

def interpolate_polynomials(matrix):
    result = list()
    for column in matrix.columns():
        polynomial = F_x.lagrange_polynomial([(index+1, values) for index, values in enumerate(column)])
        result.append(polynomial)
    return result

def inner_product_polynomials_with_witness(polys, witness):
    def mul(x, y): 
        return x * y
    def sum(x, y): 
        return x + y
    prod = map(mul, polys, witness)
    return reduce(sum, list(prod))

def inner_product_enc_polynomials_with_witness(polys, witness):
    def mul(x, y): 
        return multiply(x, int(y))
    return reduce(bn128_curve.add, map(mul, polys, witness))

def encrypt_polynomials(polynomials, encrypted_points):
    encrypted_polynomials = [None] * len(polynomials)
    for i, poly in enumerate(polynomials):
        if poly != 0:
            for j, coeff in enumerate(poly.coefficients(sparse=False)):
                encrypted_polynomials[i] = bn128_curve.add(encrypted_polynomials[i], multiply(encrypted_points[j], int(coeff)))
    return encrypted_polynomials

def encrypt_hx_tx(h_x, t_x_enc):
    h_x_enc = list()
    for index, coeff in enumerate(h_x.coefficients(sparse=False)):
        h_x_enc.append(multiply(t_x_enc[index], int(coeff)))
    return reduce(bn128_curve.add, h_x_enc)

t_x = generate_target_polynomial(len(A.rows()))
U_polys = interpolate_polynomials(A)
V_polys = interpolate_polynomials(B)
W_polys = interpolate_polynomials(C)

(encrypted_G1, encrypted_G2, t_x_enc) = trusted_setup(len(A.rows()), t_x)

poly1 = inner_product_polynomials_with_witness(U_polys, w)
poly2 = inner_product_polynomials_with_witness(V_polys, w)
poly3 = inner_product_polynomials_with_witness(W_polys, w)  

h_x, remainder = ((poly1 * poly2) - poly3).quo_rem(t_x)
assert remainder == 0, "Division has a ramainder"

hx_tx_enc = encrypt_hx_tx(h_x, t_x_enc)

A_1 = inner_product_enc_polynomials_with_witness(encrypt_polynomials(U_polys, encrypted_G1), w)
B_2 = inner_product_enc_polynomials_with_witness(encrypt_polynomials(V_polys, encrypted_G2), w)
C_1 = inner_product_enc_polynomials_with_witness(encrypt_polynomials(W_polys, encrypted_G1), w)

C_1 = bn128_curve.add(C_1, hx_tx_enc)

pairing_1 = pairing(B_2, A_1) 
pairing_2 = pairing(G2, C_1)

print(pairing_1)
print(pairing_2)

assert pairing_1 == pairing_2, "Fail"


(11333234504750252101918023511251299238232933602798776012878081055089004571850, 2609568871814267618678867373589792959124080078204745329542669313445996472770, 566745910948739961942091719345816964107197440521542437211726833759545164290, 12811774871360665709588813229542839295406508978999409831401178252357309419902, 18981747384351786441593438312446412194074051335257127020958575031216167811686, 12311155885519095680872017897916477153064386108877975375887924011188609433215, 17971280236371754363924885356159514280120898949547713391617848783810067843589, 14431499720884068542441700521347027969898985090388504678751773312711317727101, 5006862414085122068819284335529935586457773244731721386534082822182888971027, 5834133911168221276820802923951081503188647969537072337883553398118116261152, 6999901707300214622502809334103851899819829983697661250832793207382679132513, 21327438791582064962027386209035163599871401288066331460950113545149601418840)
(11333234504750252101918023511251299238232933602798776012